In [197]:
import torch
import numpy as  np
import math

from torch import nn
import torch.nn.functional as F
from torch.distributions import Normal

import tensorflow as tf

from bayes_util import standard_gaussian, gaussian_cdf, softrelu, g, delta, heavy_g, make_random_covariance_matrix

EPS = 1e-8

In [198]:
sess= tf.InteractiveSession()

/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [201]:
def standard_gaussian_torch(x):
    pi = torch.FloatTensor([math.pi]).to(x.device)
    const = 1 / torch.sqrt(2 * pi)
    return const * torch.exp(-1 / 2 * x * x)

def gaussian_cdf_torch(x):
    const = 1 / np.sqrt(2)
    return 0.5 * (1 + torch.erf(x * const))

def softrelu_torch(x):
    return standard_gaussian_torch(x) + x * gaussian_cdf_torch(x)

def heaviside_q(rho, mu1, mu2):
    """
        Computes exp( -Q(rho, mu1, mu2)) for Heaviside activation function.
        
    """
    rho_hat = torch.sqrt(1 - rho * rho)
    arcsin = torch.asin(rho)
    
    rho_s = torch.abs(rho) + EPS
    arcsin_s = torch.abs(torch.asin(rho)) + EPS
        
    A = arcsin / (2 * math.pi)
    coef = rho_s / (2 * arcsin_s * rho_hat)
    coefs_prod = (rho * rho) / (arcsin_s * rho_hat * (1 + rho_hat))
    return A * torch.exp(-(mu1*mu1 + mu2*mu2) * coef + mu1 * mu2 * coefs_prod)


def relu_q(rho, mu1, mu2):
    """
        Computes exp( -Q(rho, mu1,  mu2)) for ReLU activation function.
    """
    rho_hat_plus_one = torch.sqrt(1 - rho * rho) + 1
    g_r = torch.asin(rho) - rho / rho_hat_plus_one   # why minus?
    
    rho_s = torch.abs(rho) + EPS
    g_r_s = torch.abs(g_r) + EPS
    A = g_r / (2 * math.pi)
    
    coef_sum =  rho_s / (2 * g_r_s * rho_hat_plus_one)
    coef_prod = (torch.asin(rho) - rho) / (rho_s * g_r_s)
    return A * torch.exp(- (mu1 * mu1 + mu2 * mu2) * coef_sum + coef_prod * mu1 * mu2)


def delta_torch(rho, mu1, mu2):
    return gaussian_cdf_torch(mu1) * gaussian_cdf_torch(mu2) + relu_q(rho, mu1, mu2)

In [199]:
def get_tf_answer(function, value, mu1 = None, mu2 = None):
    value = np.array(value)
    if mu1 is  None:
        x  = tf.placeholder(tf.float32, shape = value.shape)
        f = function(x)
        return sess.run(f, feed_dict={x: value})
    else:
        mu1 = np.array(mu1)
        mu2 = np.array(mu2)
        
        rho = tf.placeholder(tf.float32, shape = value.shape)
        mu_1 = tf.placeholder(tf.float32, shape = mu1.shape)
        mu_2 = tf.placeholder(tf.float32, shape = mu2.shape)
        
        f = function(rho, mu_1, mu_2)
        return sess.run(f, feed_dict={rho: value, mu_1: mu1, mu_2: mu2})

In [200]:
rho = torch.FloatTensor([1, 0.01, -1])
mu1 = torch.FloatTensor([-1, 0, -1])
mu2 = torch.FloatTensor([2, -0.5, -1])

In [202]:
delta_torch(rho, mu1, mu2)

tensor([0.1552, 0.1550, 0.0194])

In [203]:
get_tf_answer(delta, [1, 0.01,  -1], [-1, 0, -1], [2, -0.5, -1])

array([0.15519984, 0.15497105, 0.0193752 ], dtype=float32)